# NMHH for classification

In [2]:
import sys
import os

sys.path.insert(0, '../')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
import runExperiment.hyperParameterTuning.pyNMHH.pyNMHH as pyNMHH

from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

def initialClassificationBaseLevelConfig():
    return {
    "OperatorsAndCategories":{
        "categories": ['perturbator', 'selector','refiner'],
        "operators": {
            'perturbator': ['DE', 'GA'],
            'selector': ['best'],
            'refiner': ['BayesGP']
        }
    },
    "Restrictions":{
        "CategoryTransitionRestrictions":[[0],[1,2],[0]]
    },
    "CategoryTransitionMatrix": {
        "InitialProbabilities": [1, 0.0,0.0],
        "TransitionMatrix": [
            [0.0, 0.5,0.5],
            [1.0, 0.0,0.0],
            [0.0,1.0,0.0]
        ]
    },
    "OperatorTransitionMatrices": {
        "perturbator": {
            "InitialProbabilities": [0.705285341737996, 0.29471465826200405],
            "TransitionMatrix": [
                [0.8062957200961741, 0.19370427990382585],
                [1.0, 0.0]
            ]
        },
        "selector": {
            "InitialProbabilities": [1.0],
            "TransitionMatrix": [[1.0]]
        },
        "refiner": {
            "InitialProbabilities": [1.0],
            "TransitionMatrix": [[1.0]]
        }
    },
    "OperatorParams":{
        "PerturbatorDEOperatorParams": {
                        "DE_CR": {
                            "lowerBound": 0.0,
                            "upperBound": 1.0,
                            "value": 1.0
                        },
                        "DE_FORCE": {
                            "lowerBound": 0.0,
                            "upperBound": 2.0,
                            "value": 0.566329910553018
                        }
                    },
        "PerturbatorGAOperatorParams": {
                        "GA_ALPHA": {
                            "lowerBound": 0.0,
                            "upperBound": 1.0,
                            "value": 0.0
                        },
                        "GA_CR": {
                            "lowerBound": 0.0,
                            "upperBound": 1.0,
                            "value": 0.23283148368816325
                        },
                        "GA_CR_POINT": {
                            "lowerBound": 0.0,
                            "upperBound": 1.0,
                            "value": 0.9979030839478561
                        },
                        "GA_MUTATION_RATE": {
                            "lowerBound": 0.0,
                            "upperBound": 0.5,
                            "value": 0.0
                        },
                        "GA_MUTATION_SIZE": {
                            "lowerBound": 0.0,
                            "upperBound": 100.0,
                            "value": 100.0
                        },
                        "GA_PARENTPOOL_RATIO": {
                            "lowerBound": 0.2,
                            "upperBound": 1.0,
                            "value": 0.707368716697075
                        }
                    }
    }
}

def k_fold_split(X, y, k, shuffle=True, random_state=None):
    kf = KFold(n_splits=k, shuffle=shuffle, random_state=random_state)
    
    for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        yield fold, X_train, y_train, X_test, y_test

class FlatHyperParameters:
    def __init__(self,params):
        self.params=params
        self.lowerBounds={}
        self.upperBounds={}        
        for key,value in params.items():
            if isinstance(value[0], (int, float)):
                self.lowerBounds[key]=float(value[0])
                self.upperBounds[key]=float(value[1])
            else:
                self.lowerBounds[key]=0
                self.upperBounds[key]=len(value)-1        
    def unflatten(self,flatParameters):
        original_representation = {}
        for (key, param_value), value in zip(self.params.items(), flatParameters):
            if value<self.lowerBounds[key]:
                boundedValue = self.lowerBounds[key]
            else:
                if value>self.upperBounds[key]:
                    boundedValue = self.upperBounds[key]
                else:
                    boundedValue = value 
            if isinstance(param_value[0], (int)):
                original_representation[key] = int(boundedValue)
            else:
                if isinstance(param_value[0], (float)):
                    original_representation[key] = boundedValue
                else:
                    original_representation[key] = param_value[int(boundedValue)]
        return original_representation
        
    def flatten(self,params):
        values = []
        for (key,value) in params.items():
            if isinstance(value[0], int) or isinstance(value[0], float)  :
                values.append(value)
            else:
                values.append(params[key].index(value))
        return values

    def randomUniformSample(self):
        sample = []
        for (key,bounds) in self.params.items():
            if isinstance(bounds[0], (int, float)):
                sample.append(pyNMHH.random.uniform(self.lowerBounds[key], self.upperBounds[key]))
            else:
                sample.append(pyNMHH.random.randint(self.lowerBounds[key], self.upperBounds[key]))
        return sample
    
    def flatBounds(self):
        lower = []
        upper = []
        for (key,bounds) in self.params.items():
            lower.append(self.lowerBounds[key])
            upper.append(self.upperBounds[key])
        return lower,upper
    
    def xtypes(self):
        types = []
        for (key,bounds) in self.params.items():
            if isinstance(bounds[0], (float)):
                types.append('continuous')
            else:
                types.append('discrete')
        return types
    
class TrainableModelWrapper:
    def __init__(self, X, y, fraction, classifier,predictor, classifier_name, dataset_name,hyperParamConfig,cv=3):
        self.X = X
        self.y = y
        self.fraction=fraction
        self.classifier=classifier
        self.predictor=predictor
        self.classifier_name=classifier_name
        self.dataset_name=dataset_name
        self.hyperParamConfig=hyperParamConfig
        self.cv=cv

    def __call__(self, flatHyperParams):
        return self.train_and_evaluate(self.X, self.y, self.fraction, self.classifier,self.predictor, self.hyperParamConfig.unflatten(flatHyperParams), self.classifier_name, self.dataset_name)
    
    def train_and_evaluate(self, X, y, fraction, classifier,predictor, params, classifier_name, dataset_name, num_iterations=1,foldLimit=10):
        accuracies = []
        crossvalcount=int(1/(fraction))
        for i in  range(num_iterations):
            if crossvalcount >=2:
                for fold, X_train, y_train, X_test, y_test in k_fold_split(X, y, k=crossvalcount):
                    if fold > foldLimit:
                        continue
                    print(f"                    >>>>>training: {classifier_name} with {params} on {dataset_name} at training data size fraction of {fraction} at iteration: {i+1}/{num_iterations}")
                    clf = classifier(**params)
                    clf.fit(X_test, y_test)
                    y_pred = predictor(clf,X_train)
                    accuracy = accuracy_score(y_train, y_pred)
                    accuracies.append(accuracy)
            else:
                clf = classifier(**params)
                scores = cross_val_score(clf, X, y,cv=self.cv,scoring='accuracy',n_jobs=-1)
                accuracies.append(scores.mean())
                print(f"                    >>>>>trained: {classifier_name} with {params} on {dataset_name} crossvalidationCount: {self.cv} at iteration: {i+1}/{num_iterations}-> scores:{scores}/mean{scores.mean()}")
                # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-fraction, random_state=None)
                
                # clf = classifier(**params)
                # clf.fit(X_train, y_train)
                
                # y_pred = predictor(clf,X_test)
                #     # y_pred = clf.predict(X_train)
                # accuracy = accuracy_score(y_test, y_pred)
                # accuracies.append(accuracy)
       
        return -(pyNMHH.np.array(accuracies).mean())
    
class BaseLevel():

    def __init__(self,populationSize,max_evaluations, X, y, fraction, classifier,predictor, classifier_name, dataset_name,classifierConfig,cv=3):
        self.trainableModel=TrainableModelWrapper(X, y, fraction, classifier,predictor, classifier_name, dataset_name,classifierConfig,cv)
        self.classifierConfig=classifierConfig
        self.population_size=populationSize
        self.max_evaluations=max_evaluations

    def __call__(self,baseLevelConfig, history):
        lower,upper=self.classifierConfig.flatBounds()
        types=self.classifierConfig.xtypes()
        wrapped_objective = pyNMHH.FunctionWrapper( self.trainableModel, self.max_evaluations,1000000,lowerbounds=lower,upperbounds=upper,xtypes=types)
        population = [pyNMHH.Individual(self.classifierConfig.randomUniformSample()) for _ in range(self.population_size)]
        # prevsize=min(population_size,len(history.best_offspring))
        # population = [Individual(self.classifierConfig.randomUniformSample()) for _ in range(population_size-prevsize)]
        # if (len(history.best_offspring)>0):
        #     [population.append(Individual(prev)) for prev in history.best_offspring[-prevsize:]]
        # else:
        #     population.append(Individual(self.classifierConfig.randomUniformSample()))
        # try:
        best_solution, best_fitness, total_evaluations = pyNMHH.NMHHBaseOpt(wrapped_objective, population, self.max_evaluations, baseLevelConfig, history)
        return {"fitness":best_fitness,"solution":best_solution}
        # except Exception as e:
        #     print(f"Error in optimization: {e}")
        #     return float('inf')  # Return a high value if there's an error
    


## Run classification experiment

In [3]:
def runClassificationTuning(config):
    print(f"            >>>pyNMHH optimization starting for {config['classifierName']}-{config['datasetName']}")
    best_params, best_fitness, history,bestSequence,best_solution = pyNMHH.NMHHHyperOpt(config['baseLevelConfig'], 
                                                    BaseLevel(config['populationSize'],config['baselevelIterations'],
                                                              config['X'],config['Y'],config['trainingFraction'], 
                                                              config['classifier'],lambda clf,train: clf.predict(train),config['classifierName'], 
                                                              config['datasetName'], config['flatHyperParameters'],config['crossValidations']), 
                                                    iterations=config['pyNMHHSteps'],
                                                    initialTemp=config['SA_temp0'], cooling_rate= config['SA_coolingRate'])

    print(f"            >>>Best fitness achieved: {best_fitness}")
    # print(f"Best {classifierName} hyperparameter history")
    # [print(f'Best hyperparameter step: {rvParamFlattener.unflatten(flatsvmparam)} -> frac: {frac}={accuracy} vs big: {totalSVMScore(rvParamFlattener.unflatten(flatsvmparam),X,y)} ') for flatsvmparam,accuracy in zip(history.best_offspring,history.function_values)]
    # print("Optimized baseLevelConfig parameters:")
    # print(pyNMHH.json.dumps(best_params, indent=2))
    print(f"            >>>best sequence: {bestSequence}" )
    print(f'            >>>tuned  hyperparameters {config["flatHyperParameters"].unflatten(best_solution)}')
    return {
                "bestBaseConfig": best_params,
                "bestAccuracy":-best_fitness,
                'solution':config["flatHyperParameters"].unflatten(best_solution),
                'bestSequence':bestSequence,
            }
    

### SVM

In [4]:

from timeit import default_timer as timer
import numpy as np

def runClassificationTuningExperiment(config):
    solutions=[]
    experimentTimes=[]
    for i in range(config['classificationTuningCount']):
        print(f'            >>>Running  iteration {i+1}/{config["classificationTuningCount"]}\n')
        start = timer()
        solutions.append(runClassificationTuning(config))
        end = timer()
        elapsed=end-start
        experimentTimes.append(elapsed)
        etaSeconds=(config['classificationTuningCount']-(i+1))*np.mean(experimentTimes)
        print(f'            >>>Ran  {i+1}/{config["classificationTuningCount"]} iterations elapsed seconds {elapsed} eta: {etaSeconds} seconds')
    return solutions

In [5]:
def runSVMClassification():
    svm_params = {
        'C': [1.0,50.0],
        "kernel":['linear','poly','rbf','sigmoid']
    }
    digits=datasets.load_digits()
    X,y=digits.data, digits.target
    config={
        'X':X,
        'Y':y,
        'flatHyperParameters':FlatHyperParameters(svm_params),
        'classifier':svm.SVC,
        'populationSize':5,
        'baselevelIterations':10,
        'crossValidations':3,
        'pyNMHHSteps': 3,
        'classificationTuningCount':7,
        'classifierName':'SVM',
        'datasetName':'Digits',
        'trainingFraction':0.75,
        'baseLevelConfig':initialClassificationBaseLevelConfig(),
        'SA_temp0':1.0,
        'SA_coolingRate':0.995
    }
    runClassificationTuningExperiment(config)
   
    
# print(runSVMClassification())

### Random Forest

In [25]:

def runRFClassification():
    rf_params = {
        'n_estimators': [10,100],
        "max_features":[1,64],
        'max_depth': [5,50],
        "min_samples_split":[2,11],
        "min_samples_leaf":[1,11],
        "criterion":['gini','entropy']
    }
    digits=datasets.load_digits()
    X,y=digits.data, digits.target
    config={
        'X':X,
        'Y':y,
        'flatHyperParameters':FlatHyperParameters(rf_params),
        'classifier':RandomForestClassifier,
        'populationSize':10,
        'baselevelIterations':50,
        'pyNMHHSteps': 3,
        'crossValidations':3,
        'classificationTuningCount':10,
        'classifierName':'RandomForest',
        'datasetName':'Digits',
        'trainingFraction':0.75,
        'baseLevelConfig':initialClassificationBaseLevelConfig(),
        'SA_temp0':1.0,
        'SA_coolingRate':0.995
    }
    runClassificationTuningExperiment(config)
     
# runRFClassification()